In [ ]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

In [ ]:
!rm -rf /content/data

import zipfile

zip_path = "/content/drive/MyDrive/nsfw_model/original.zip"
extract_path = "/content/original"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset restored ✔️")


In [ ]:
import os
from PIL import Image

# Input and output directories
INPUT_DIR = "original"
OUTPUT_DIR = "processed"

IMAGE_SIZE = (224, 224)
SUPPORTED_FORMATS = (".jpg", ".jpeg", ".png", ".webp")

os.makedirs(OUTPUT_DIR, exist_ok=True)

for category in ["nsfw", "sfw"]:
    input_category_path = os.path.join(INPUT_DIR, category)
    output_category_path = os.path.join(OUTPUT_DIR, category)
    os.makedirs(output_category_path, exist_ok=True)

    for subfolder in os.listdir(input_category_path):
        input_sub_path = os.path.join(input_category_path, subfolder)
        output_sub_path = os.path.join(output_category_path, subfolder)
        os.makedirs(output_sub_path, exist_ok=True)

        for img_name in os.listdir(input_sub_path):
            if img_name.lower().endswith(SUPPORTED_FORMATS):
                try:
                    img_path = os.path.join(input_sub_path, img_name)
                    img = Image.open(img_path).convert("RGB")
                    img = img.resize(IMAGE_SIZE)

                    save_path = os.path.join(output_sub_path, img_name)
                    img.save(save_path)

                except Exception as e:
                    print(f"Error processing {img_name}: {e}")

print("✅ Image resizing completed successfully!")


In [ ]:
from torchvision.datasets import ImageFolder

dataset = ImageFolder("processed")
print(dataset.class_to_idx)


In [ ]:
from torchvision.datasets import ImageFolder
from collections import Counter

dataset = ImageFolder(root="processed")

# Total images
print("Total images:", len(dataset))

# Class names and indices
print("Class to index mapping:", dataset.class_to_idx)

# Count images per class
labels = [label for _, label in dataset]
class_counts = Counter(labels)

for class_idx, count in class_counts.items():
    class_name = dataset.classes[class_idx]
    print(f"{class_name}: {count}")


In [ ]:
import os
import shutil

def get_unique_name(dst_dir, filename):
    name, ext = os.path.splitext(filename)
    counter = 1
    new_name = filename

    while os.path.exists(os.path.join(dst_dir, new_name)):
        new_name = f"{name}_{counter}{ext}"
        counter += 1

    return new_name


for sub in os.listdir(category_path):
    sub_path = os.path.join(category_path, sub)
    if os.path.isdir(sub_path):
        for img in os.listdir(sub_path):
            src = os.path.join(sub_path, img)
            new_name = get_unique_name(category_path, img)
            dst = os.path.join(category_path, new_name)
            shutil.move(src, dst)


In [ ]:
from torchvision.datasets import ImageFolder

dataset = ImageFolder("processed")
print(dataset.class_to_idx)


In [ ]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

dataset = ImageFolder(root="processed")

total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size   = int(0.15 * total_size)
test_size  = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size]
)

train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform   = val_test_transform
test_dataset.dataset.transform  = val_test_transform


In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=2)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=2)

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=2)


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)

# Replace final layer
model.fc = nn.Linear(model.fc.in_features, 2)

model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = correct / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = val_correct / val_total

    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Loss: {running_loss:.4f} "
          f"Train Acc: {train_acc:.4f} "
          f"Val Acc: {val_acc:.4f}")


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())

print("Accuracy :", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred))
print("Recall   :", recall_score(y_true, y_pred))
print("F1-score :", f1_score(y_true, y_pred))


In [ ]:
MODEL_NAME = "resnet50_nsfw_model.pt"

torch.save(model.state_dict(), MODEL_NAME)

print(f"✅ Model saved successfully as {MODEL_NAME}")


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 2)

model.load_state_dict(torch.load("resnet50_nsfw_model.pt", map_location=device))
model = model.to(device)
model.eval()


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
def predict_from_url(image_url, threshold=0.8):
    try:
        response = requests.get(image_url, timeout=10)
        image = Image.open(BytesIO(response.content)).convert("RGB")

        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(image)
            probabilities = torch.softmax(outputs, dim=1)
            nsfw_prob = probabilities[0][1].item()

        if nsfw_prob >= threshold:
            return {
                "prediction": "NSFW",
                "confidence": round(nsfw_prob, 4)
            }
        else:
            return {
                "prediction": "SFW",
                "confidence": round(1 - nsfw_prob, 4)
            }

    except Exception as e:
        return {"error": str(e)}


In [ ]:
def predict_from_url(image_url, threshold=0.8):
    try:
        response = requests.get(image_url, timeout=10)
        response.raise_for_status()

        image = Image.open(BytesIO(response.content)).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)

        model.eval()
        with torch.no_grad():
            outputs = model(image)
            probabilities = torch.softmax(outputs, dim=1)

            nsfw_prob = probabilities[0][0].item()  # 0 = NSFW
            sfw_prob  = probabilities[0][1].item()  # 1 = SFW

        prediction = "NSFW" if nsfw_prob >= threshold else "SFW"

        return {
            "prediction": prediction,
            "nsfw_score": round(nsfw_prob, 4),
            "sfw_score": round(sfw_prob, 4),
            "threshold": threshold
        }

    except Exception as e:
        return {"error": str(e)}


In [ ]:
image_url = "https://imgnew.outlookindia.com/uploadimage/library/16_9/16_9_5/IMAGE_1684860404.jpg"
result = predict_from_url(image_url)
print(result)


In [ ]:
image_url = "https://th.bing.com/th/id/OIP.lTVRqQRf7Bq7UiX2zMiK3QHaE8?o=7rm=3&rs=1&pid=ImgDetMain&o=7&rm=3"
result = predict_from_url(image_url)
print(result)


In [ ]:
image_url = "https://www.infiniterecovery.com/wp-content/uploads/2024/09/shutterstock_2475282919-1.jpg"
result = predict_from_url(image_url)
print(result)



In [ ]:
image_url = "https://imgnew.outlookindia.com/uploadimage/library/16_9/16_9_5/IMAGE_1684860404.jpg"
result = predict_from_url(image_url)
print(result)


In [ ]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())


In [ ]:
cm = confusion_matrix(y_true, y_pred)
print(cm)


In [ ]:
from google.colab import files
files.download("resnet50_nsfw_model.pt")
